In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [3]:
# counties listed should have the same years available
start = time.time()
Lake_taxlot = combine_taxlot(yearstart=2011, yearend=2017, all_counties=False, counties=['Lake'])
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

2011
2012
2013
2014
2015
2016
it took 0.0 minutes to complete


In [4]:
Marion_taxlot = combine_taxlot(yearstart=2009, yearend=2017, skips=[2010, 2013], all_counties=False, counties=['Marion'])

2009
2011
2012
2014
2015
2016


In [5]:
all_taxlot = Lake_taxlot.append(Marion_taxlot)

In [48]:
all_taxlot.columns

Index(['year', 'county', 'ORTaxlot', 'geometry'], dtype='object')

In [6]:
all_taxlot.shape

(837009, 4)

In [7]:
n=8

In [8]:
wd_df = get_all_wd(num=n-1)

In [9]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [10]:
wd_df.shape

(5402, 26)

In [11]:
setID = 'Set00'+str(n); nm_to_add = wd_df.shape[0]

In [12]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [13]:
wd.county.unique()

array(['Lake', 'LAKE', 'Marion', 'MARION'], dtype=object)

In [14]:
wd.loc[:, 'county'] = wd.county.apply(lambda x: x.capitalize())

In [15]:
wd.groupby(['county'])['wetdet_delin_number'].nunique()

county
Lake       33
Marion    262
Name: wetdet_delin_number, dtype: int64

In [16]:
wd.trsqq.apply(lambda x: scan_trsqq(x))

0      0
1      0
2      0
3      0
4      0
      ..
454    0
455    0
456    0
457    0
458    0
Name: trsqq, Length: 459, dtype: int64

In [51]:
wdf, setgdf, unmatched_df = run_Tier1(setID=setID, nm_to_add=nm_to_add, all_taxlot=all_taxlot)

it took 3.0 minutes to complete
it is about 7.19% of data in the original 459 records unmatched
there are 1 records (0.22% of the original records) without parcel id


In [40]:
wdf.shape

(459, 25)

In [52]:
unmatched_df.shape

(33, 25)

In [53]:
len(unmatched_df.wetdet_delin_number.unique())

19

In [21]:
setgdf.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'coord_src', 'doc_name',
       'doc_link', 'isbatfile', 'status_nm', 'receiveddt', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'record_ID', 'notes', 'recyear',
       'IDyear', 'missinglot', 'lots', 'lot', 'cnt_code', 'ORTaxlot',
       'geometry'],
      dtype='object')

In [22]:
setgdf.shape

(831, 30)

In [23]:
len(setgdf.wdID.unique())

276

In [54]:
unmatched_df.missinglot.unique()

array(['Y', 'N', nan], dtype=object)

In [55]:
r1_df, r2_df = run_Tier2_step1(setID=setID, unmatched_df=unmatched_df, all_taxlot=all_taxlot)

reviewing WD2011-0023
all lots are matched...
corrected trsqq...
reviewing WD2010-0305
all lots are matched...
corrected trsqq...
reviewing WD2010-0015
all lots are matched...
corrected trsqq...
reviewing WD2008-0098
all lots are matched...
corrected trsqq...
reviewing WD2012-0175
some lots are not matched, need to review trsqq, the close-match is 09S03E30DA
lots to check: ['100', '102', '3600'], and lots to compare: ['100', '200', '300', '400', '500', '600', '700', '1100', '1300', '1400', '1500', '1600', '1700', '1800', '2000', '2100', '2200', '2300', '2400', '2500', '2700', '2701', '2800', '3000', '3400', '3500', '3600', '3700', '3800', 'ROADS', 'WATER']
reviewing WD2013-0362
coordinate might be incorrect, nearby taxlot is 2410.00S03.00W2400--0000WATER, about 27313 ft away


In [58]:
matched, unmatched_df2 = run_Tier2_step3(r1_df=r1_df, r2_df=r2_df, setID=setID, nm_to_add=nm_to_add, 
                                        wd=wdf, all_taxlot=all_taxlot)

it is about 5.66% of data in the original 459 records unmatched
there are 1 records (0.22% of the original records) without parcel id


In [56]:
review_loop_r1(df=unmatched_df2, partial=True, idx=True)

0.0% digitized, 19 records remained, expected to be done in about 5 hours...
WD2011-0335
index = 248
     county       trsqq parcel_id   latitude   longitude  record_ID notes  \
247  Marion   05S02W12C      MANY  45.151031 -122.880669       5650  Many   
245  Marion  05S01W07CB      MANY  45.151031 -122.880669       5648  Many   
244  Marion  05S02W12BB      MANY  45.151031 -122.880669       5647  Many   
250  Marion   05S02W12A      MANY  45.151031 -122.880669       5653  Many   
248  Marion  05S02W12BC      MANY  45.151031 -122.880669       5651  Many   
246  Marion  05S02W12DB      MANY  45.151031 -122.880669       5649  Many   

    missinglot status_name is_batch_file  
247          Y    Approved             N  
245          Y    Approved             N  
244          Y    Approved             N  
250          Y    Approved             N  
248          Y    Approved             N  
246          Y    Approved             N  
Press 'p' to pause or any key to stop...s
